In [1]:
import os
import warnings
import pickle
import numpy as np
import random

os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore')

In [2]:
with open(r"./Data/dataset_n306_X_500_12.pickle", "rb") as input_file:
    dataset = pickle.load(input_file)

(x_dataset, y_dataset, labels) = dataset
permutation = list(range(x_dataset.shape[0]))

# if you need to shuffle the dataset
random.shuffle(permutation)

x_dataset = np.array([x_dataset[i] for i in permutation])
y_dataset = np.array([y_dataset[i] for i in permutation])
labels = np.array([labels[i] for i in permutation])

leads = [6, 7, 8] # V1 V2 V3
n_leads = len(leads)

x_dataset = np.take(x_dataset, leads, axis=-1)

In [ ]:
from search import double_k_fold, search_direct, k_fold_val_reg
from generators import grid_hp_generator
from Models.model_buiders import build_ESN

m_list = double_k_fold(x_dataset, y_dataset, n_out_fold=5, n_in_fold=4, n_restart=3,
                       hp_generator=grid_hp_generator,
                       n_iter=108, #if grid_hp_generator is used this value is ignored
                       model_builder=build_ESN,
                       search=search_direct, val_fun=k_fold_val_reg,
                       units=[500, 2000], input_scaling=[1.5],
                       bias_scaling=[0.1],
                       spectral_radius=[0.999], leaky=[0.1])




In [7]:
train_accuracys = []
val_accuracys = []
test_accuracys = []
sens_accuracys = []
specs_accuracys = []
for mf in m_list:
    trains = np.array([m[2] for m in mf])
    vals = np.array([m[3] for m in mf])
    tests = np.array([m[4][0] for m in mf])
    sens = np.array([m[4][1] for m in mf])
    specs = np.array([m[4][2] for m in mf])
    train_accuracys.append(trains.mean())
    val_accuracys.append(vals.mean())
    test_accuracys.append(tests.mean())
    sens_accuracys.append(sens.mean())
    specs_accuracys.append(specs.mean())

train_accuracys = np.array(train_accuracys)
val_accuracys = np.array(val_accuracys)
test_accuracys = np.array(test_accuracys)
sens_accuracys = np.array(sens_accuracys)
specs_accuracys = np.array(specs_accuracys)

final_train_avg = train_accuracys.mean()
final_train_std = train_accuracys.std()
final_val_avg = val_accuracys.mean()
final_val_std = val_accuracys.std()
final_test_avg = test_accuracys.mean()
final_test_std = test_accuracys.std()
final_sens_avg = sens_accuracys.mean()
final_sens_std = sens_accuracys.std()
final_spec_avg = specs_accuracys.mean()
final_spec_std = specs_accuracys.std()

with open(fr"./result_ESN_{n_leads}.txt", "w") as output_file:
    print("leads: ", leads, file=output_file)
    print("train accuracy: ", final_train_avg, file=output_file)
    print("train accuracy std: ", final_train_std, file=output_file)
    print("validation accuracy: ", final_val_avg, file=output_file)
    print("validation accuracy std: ", final_val_std, file=output_file)
    print("test accuracy: ", final_test_avg, file=output_file)
    print("test accuracy std: ", final_test_std, file=output_file)
    print("sensitivity accuracy: ", final_sens_avg, file=output_file)
    print("sensitivity accuracy std: ", final_sens_std, file=output_file)
    print("specificity accuracy: ", final_spec_avg, file=output_file)
    print("specificity accuracy std: ", final_spec_std, file=output_file)

    print(
        f"${round(final_train_avg, 3)} \pm {round(final_train_std, 4)}$ & ${round(final_val_avg, 3)} \pm {round(final_val_std, 4)}$ & ${round(final_test_avg, 3)} \pm {round(final_test_std, 4)}$ & ${round(final_sens_avg, 3)} \pm {round(final_sens_std, 4)}$ & ${round(final_spec_avg, 3)} \pm {round(final_spec_std, 4)}$",
        file=output_file)
    print("", file=output_file)
    print("units:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["units"], file=output_file)
    print("", file=output_file)
    print("input scaling:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["input_scaling"], file=output_file)
    print("", file=output_file)
    print("bias scaling:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["bias_scaling"], file=output_file)
    print("", file=output_file)
    print("spectral radius:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["spectral_radius"], file=output_file)
    print("", file=output_file)
    print("leaking rate:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["leaky"], file=output_file)
    print("", file=output_file)
    print("reg:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[1]["reg"], file=output_file)
    print("", file=output_file)
    print("training acc:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[2], file=output_file)
    print("", file=output_file)
    print("val acc:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[3], file=output_file)
    print("", file=output_file)
    print("test acc:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[4][0], file=output_file)
    print("", file=output_file)
    print("sens acc:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[4][1], file=output_file)
    print("", file=output_file)
    print("spec acc:", file=output_file)
    for mf in m_list:
        for m in mf:
            print(m[4][2], file=output_file)